In [8]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

ml_client = MLClient.from_config(credential=credential)

workspace = ml_client.workspaces.get(name=ml_client.workspace_name)

print(ml_client.workspace_name, workspace.resource_group, workspace.location, ml_client.connections._subscription_id, sep = '\n')

Register existing or Create model (from Run, or from Model File)

In [11]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

# job_name = returned_job.name
job_name = "gentle_plate_bskd49s6gn"

model = Model(
    path=f"azureml://jobs/{job_name}/outputs/artifacts/paths/model/",
    name="mlflow-diabetes-02",
    description="Model created from run.",
    type=AssetTypes.MLFLOW_MODEL,
)

# model = Model(
#     name="model-endpoints",
#     path="./model-endpoints",
#     type=AssetTypes.MLFLOW_MODEL,
#     description="test model for online endpoint",
# )

# Uncomment after adding required details above
ml_client.models.create_or_update(model)

### 1. Create Batch endpoint

In [ ]:
import datetime
from azure.ai.ml.entities import BatchEndpoint

endpoint_name = "batch-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create a batch endpoint
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A batch endpoint for classifying diabetes in patients",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint)

### 2. Create Batch Deployment

In [13]:
from azure.ai.ml.entities import Environment, BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import BatchDeploymentOutputAction

# Get register model by name
latest = next(iter(ml_client.models.list(name="mlflow-diabetes")))
print(latest)

deployment = BatchDeployment(
    name="classifier-diab-mdl",
    description="A diabetes model classifier batch endp",
    endpoint_name=endpoint.name,
    model=f"azureml:{latest.name}:{latest.version}",  # <-- reference existing Model
    compute="ml-compute-ntb",
    instance_count=1,
    max_concurrency_per_instance=2,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)
ml_client.batch_deployments.begin_create_or_update(deployment)

In [4]:
endpoint.defaults = {}

endpoint.defaults["deployment_name"] = deployment.name

ml_client.batch_endpoints.begin_create_or_update(endpoint)

Create Data Asset If not created. Or Reuse existing one if created.

In [9]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path = "./data"
dataset_name = "test-data-new"

# patient_dataset_unlabeled = Data(
#     path=data_path,
#     type=AssetTypes.URI_FOLDER,
#     description="An unlabeled dataset for diabetes classification",
#     name=dataset_name,
# )
# ml_client.data.create_or_update(patient_dataset_unlabeled)

# Data Asset from file

csv_path = "./data/test-data-01.csv"
asset = Data(
    name="diabetes-infer-file",
    path=csv_path,
    type=AssetTypes.URI_FILE,
    description="Single CSV for batch scoring",
)
ml_client.data.create_or_update(asset)

In [14]:
# Retreive Endpoint and Deployment 

endpoint = ml_client.batch_endpoints.get("batch-10131615728401")

print(endpoint)

deployment01 = ml_client.batch_deployments.get("classifier-diab-mdl", endpoint.name)

print(deployment01)

In [15]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

# get data asset
data_asset = ml_client.data.get(name="diabetes-infer-file", version="1")

# pass data asset name and version (Manualy or automaticaly)
input = Input(type=AssetTypes.URI_FILE, path="azureml:diabetes-infer-file:1")

job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, 
    input=input)

In [18]:
# check the results
import pandas as pd

ml_client.jobs.download(name=job.name, download_path=".", output_name="score")

score = pd.read_csv("predictions.csv", index_col=0, names=["patient", "prediction", "file"])
score

In [12]:
# Data Asset Test. Read data asset if it was created properly.

from pathlib import Path
import pandas as pd

import mltable


data_asset = ml_client.data.get(name="diabetes-infer-file", version="1")

print("ID:", data_asset.id)
print("Type:", data_asset.type)
print("Path:", data_asset.path)
print("Version:", data_asset.version)

df = pd.read_csv(data_asset.path)
df


## Online Endpoint


In [20]:
# Step 1 Deploy Endpoint

from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for MLflow diabetes model",
    auth_mode="key",
)

ml_client.begin_create_or_update(endpoint).result()

In [17]:
# Step 2 - Deploy models and depoloyments or reuse exissting models

from azure.ai.ml.entities import Model, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes

# 2.1  create/register model if needed 
# model = Model(
#     name="model-endpoints",
#     path="./model-endpoints",
#     type=AssetTypes.MLFLOW_MODEL,
#     description="test model for online endpoint",
# )

# ml_client.models.create_or_update(model)


# 2.2 Get existing(already registered) model

latest = next(iter(ml_client.models.list(name="model-endpoints")))
# print(latest)

# Get model is aleady exists
online_endpoint = ml_client.online_endpoints.get("endpoint-10221045651804")
print(online_endpoint.name)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint.name,
    model=f"azureml:{latest.name}:{latest.version}",
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

# blue deployment takes 100 traffic
# endpoint.traffic = {"blue": 100}
# ml_client.begin_create_or_update(endpoint).result()

In [11]:
# Test Online Endpoint 

# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name="endpoint-10221045651804",
    deployment_name="blue",
    request_file="test-enpdata.json",
)

print(response)

if response[1]=='1':
    print("Diabetic")
else:
    print ("Not diabetic")